In [46]:
from langchain_community.document_loaders import TextLoader
loader = TextLoader("speech.txt")
text_documents= loader.load()
text_documents

[Document(metadata={'source': 'speech.txt'}, page_content='In the last two years, India has seen a dramatic drop in Global democratic rankings. The V- Dem Institute’s Democracy Report 2022 ranked India 93rd out of 179 countries in the Liberal Democracy Index, classifying India as an “electoral autocracy.”\n\nWhereas the 2021 Democracy Index published by the Economist Intelligence Unit (EIU) classified India as a ‘flawed democracy’ with a score of 6.91, an increase over India’s score of 6.61 on the Democracy Index 2020, but ranked 46 out of 167 countries, trailing behind countries such as South Africa, Botswana, and the Czech Republic.\n\nBut how credible are these rankings? It is difficult to evaluate because  a democratic system of governance has many administrative and political aspects that could vary from one country to the other. As a result, understanding the parameters used by the aforementioned studies is critical to fully comprehend the significance of these rankings.\n\nInter

In [47]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=100,
    chunk_overlap=20,
    length_function=len,
    is_separator_regex=False,
)
documents = text_splitter.split_documents(text_documents)

In [48]:
from langchain_community.embeddings import OllamaEmbeddings
embeddings=(
    OllamaEmbeddings(model="llama3.2:1b")  ##by default it ues llama2
)


In [49]:
embeddings


OllamaEmbeddings(base_url='http://localhost:11434', model='llama3.2:1b', embed_instruction='passage: ', query_instruction='query: ', mirostat=None, mirostat_eta=None, mirostat_tau=None, num_ctx=None, num_gpu=None, num_thread=None, repeat_last_n=None, repeat_penalty=None, temperature=None, stop=None, tfs_z=None, top_k=None, top_p=None, show_progress=False, headers=None, model_kwargs=None)

In [50]:
from langchain_community.vectorstores import FAISS

 # Assuming 'documents' from CELL INDEX: 2 is what you want to use
vectorstore = FAISS.from_documents(documents, embeddings)
vectorstore

In [55]:
# pip install -U langchain langchain-community

from langchain_community.chat_models import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_messages(
    [("system", "consider youself as an expert of democracy:\n\n{context}")]
)
llm = ChatOllama(model="llama3.2:1b")
chain = create_stuff_documents_chain(llm, prompt)

chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='consider youself as an expert of democracy:\n\n{context}'), additional_kwargs={})])
| ChatOllama(model='llama3.2:1b')
| StrOutputParser(), kwargs={}, config={'run_name': 'stuff_documents_chain'}, config_factories=[])

In [56]:
from langchain.chains import create_retrieval_chain


retriever=vectorstore.as_retriever()
retrieval_chain=create_retrieval_chain(retriever,chain)
retrieval_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x11ed807d0>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='consider youself as an expert of democracy:\n\n{context}'), additional_kwargs={})])
            | ChatOllama(model='llama3.2:1b')
            | StrOutputPa

In [58]:
result=retrieval_chain.invoke({"input":"give 5 maximum repeating words from artcle?"})

print(result.get("answer"))

<|start_header_id|>assistant<|end_header_id|>

It seems like you're referring to a report or study that critiques India's democratic performance and highlights areas where it may be struggling. I'll provide some context and possible implications, while also acknowledging the complexities involved in evaluating democracy.

**Critique of Democracy:**

1. **Erosion of Civil Liberties**: The statement suggests that India's democratic institutions have been compromised, leading to a decline in civil liberties. This could be due to various factors such as increased surveillance, censorship, and restrictions on free speech.
2. **Perceived Autocratic Tendencies**: Your mention of using government power to push forward autocratic agendas implies concerns about the concentration of power, potential disregard for democratic processes, and the erosion of checks and balances.

**Implications:**

1. **Constitutional Framework**: The ranking of India's democracy score could highlight weaknesses in it